In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import pandas as pd
import numpy as np
from bayesian.train_bn import structure_learning, parameter_learning, parameter_learning_mix
from preprocess.discretization import get_nodes_type, discretization, inverse_discretization, code_categories, get_nodes_sign
from bayesian.save_bn import save_structure, save_params, read_structure, read_params
from bayesian.sampling import generate_synthetics
from external.libpgm.hybayesiannetwork import HyBayesianNetwork
from visualization.visualization import draw_BN
from bayesian.calculate_accuracy import calculate_acc
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from copy import copy
from external.libpgm.sampleaggregator import SampleAggregator
import operator
from sklearn.metrics import accuracy_score, mean_squared_error
from scipy import stats
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
import math
from pomegranate import DiscreteDistribution
import random
from sklearn.model_selection import train_test_split

In [3]:
geo = geo = pd.read_csv('../data/hackathon_processed.csv')
columns = ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Hydrocarbon type', 'Gross','Netpay','Porosity','Permeability', 'Depth']
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)

In [4]:
geo_types = get_nodes_type(geo)
geo_types

{'Tectonic regime': 'disc',
 'Period': 'disc',
 'Lithology': 'disc',
 'Structural setting': 'disc',
 'Hydrocarbon type': 'disc',
 'Gross': 'cont',
 'Netpay': 'cont',
 'Porosity': 'cont',
 'Permeability': 'cont',
 'Depth': 'cont'}

In [5]:
geo_signs = get_nodes_sign(geo)
geo_signs

{'Gross': 'pos',
 'Netpay': 'pos',
 'Porosity': 'pos',
 'Permeability': 'pos',
 'Depth': 'pos'}

In [6]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [8]:
geo_coded, label_coder = code_categories(geo, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'equal_frequency', columns_for_disc)
geo_only_discrete, discrete_coder = discretization(geo, 'equal_frequency', columns_for_disc)

In [13]:
bn = structure_learning(geo_discrete, 'HC', geo_types, 'K2')
param = parameter_learning(geo, geo_types, bn)

  0%|          | 11/1000000 [00:01<41:30:46,  6.69it/s]


In [14]:
save_structure(bn, 'full_net')
skel = read_structure('full_net')
save_params(param, 'full_net_param')
params = read_params('full_net_param')
full = HyBayesianNetwork(skel, params)

In [15]:
sample = generate_synthetics(full, geo_signs)

In [16]:
sample

,Hydrocarbon type,Lithology,Tectonic regime,Structural setting,Period,Porosity,Permeability,Depth,Gross,Netpay
0,OIL,SANDSTONE,INVERSION,INVERSION,NEOGENE,25.358711,1210.441529,1603.860675,2505.018558,391.321764
1,OIL,DOLOMITE,COMPRESSION,INTRACRATONIC,CARBONIFEROUS,15.142450,314.009127,4.665801,44.862402,113.866799
2,OIL,SANDSTONE,INVERSION,SUB-SALT,PERMIAN,10.446009,1888.011576,3310.394888,111.846804,68.818743
3,OIL,SANDSTONE,INVERSION,INVERSION,JURASSIC,25.234426,737.598331,1452.639398,276.000000,77.186374
4,OIL,SANDSTONE,COMPRESSION,FORELAND,CARBONIFEROUS,16.857232,1250.959425,875.526854,57.260493,50.031014
...,...,...,...,...,...,...,...,...,...,...
995,OIL,SANDSTONE,INVERSION,INVERSION,NEOGENE,26.959299,1716.617363,441.010063,1703.891627,194.075604
996,OIL,SANDSTONE,EXTENSION,PASSIVE MARGIN,PALEOGENE,21.411637,164.356235,725.630744,639.900000,123.032237
997,OIL,SANDSTONE,COMPRESSION,SALT,JURASSIC,14.189824,60.130537,4603.330343,242.514863,13.617499
998,GAS-CONDENSATE,LIMESTONE,COMPRESSION,INTRACRATONIC,JURASSIC,14.273777,145.048486,1419.161734,22.500000,42.518737
